In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from mlxtend.frequent_patterns import apriori, association_rules
from sklearn import cluster
from sklearn.preprocessing import LabelBinarizer
import warnings
warnings.filterwarnings('ignore')
from sklearn.feature_selection import mutual_info_classif, f_classif, SelectKBest
sns.set(style='darkgrid')
pd.options.display.max_columns = None

In [2]:
green_data = pd.read_csv('../green_consensus.csv')
hinselmann_data = pd.read_csv('../hinselmann_consensus.csv')
schiller_data = pd.read_csv('../schiller_consensus.csv')

experts = ['experts::{}'.format(i) for i in range(5)]
#target_attrs.append('consensus')

green = green_data.drop(columns=experts)
hin = hinselmann_data.drop(columns=experts)
sch = schiller_data.drop(columns=experts)
col = pd.concat([green, hin, sch])

col.head()

,cervix_area,os_area,walls_area,speculum_area,artifacts_area,cervix_artifacts_area,os_artifacts_area,walls_artifacts_area,speculum_artifacts_area,cervix_specularities_area,os_specularities_area,walls_specularities_area,speculum_specularities_area,specularities_area,area_h_max_diff,rgb_cervix_r_mean,rgb_cervix_r_std,rgb_cervix_r_mean_minus_std,rgb_cervix_r_mean_plus_std,rgb_cervix_g_mean,rgb_cervix_g_std,rgb_cervix_g_mean_minus_std,rgb_cervix_g_mean_plus_std,rgb_cervix_b_mean,rgb_cervix_b_std,rgb_cervix_b_mean_minus_std,rgb_cervix_b_mean_plus_std,rgb_total_r_mean,rgb_total_r_std,rgb_total_r_mean_minus_std,rgb_total_r_mean_plus_std,rgb_total_g_mean,rgb_total_g_std,rgb_total_g_mean_minus_std,rgb_total_g_mean_plus_std,rgb_total_b_mean,rgb_total_b_std,rgb_total_b_mean_minus_std,rgb_total_b_mean_plus_std,hsv_cervix_h_mean,hsv_cervix_h_std,hsv_cervix_s_mean,hsv_cervix_s_std,hsv_cervix_v_mean,hsv_cervix_v_std,hsv_total_h_mean,hsv_total_h_std,hsv_total_s_mean,hsv_total_s_std,hsv_total_v_mean,hsv_total_v_std,fit_cervix_hull_rate,fit_cervix_hull_total,fit_cervix_bbox_rate,fit_cervix_bbox_total,fit_circle_rate,fit_circle_total,fit_ellipse_rate,fit_ellipse_total,fit_ellipse_goodness,dist_to_center_cervix,dist_to_center_os,consensus
0,0.344647,0.003080,0.047522,0.288216,0.178585,0.016564,0.0,0.043500,0.010149,0.000133,0.0,0.000000,0.085833,0.024907,0.263560,37.594458,15.785021,21.809437,53.379479,109.918445,38.735421,71.183024,148.653865,55.029618,22.160330,32.869287,77.189948,38.561367,38.119059,0.442308,76.680426,95.109755,51.565052,43.544702,146.674807,48.808474,40.765228,8.043247,89.573702,5.014628,2.991944,167.952780,25.813163,109.919447,38.733741,5.090801,2.936650,159.486916,38.437294,95.123889,51.583029,0.923067,0.373371,0.844454,0.408130,0.603399,0.571175,0.962995,0.357890,85.474311,0.265933,0.346294,1.0
1,0.165329,0.000000,0.048236,0.504736,0.502783,0.007012,0.0,0.097405,0.973837,0.004055,0.0,0.000000,0.054999,0.028431,0.000000,59.505882,24.361877,35.144005,83.867760,122.366075,44.742407,77.623669,167.108482,78.058434,30.818729,47.239706,108.877163,54.932467,39.447415,15.485052,94.379883,101.680459,46.028852,55.651607,147.709311,63.218931,43.925912,19.293019,107.144843,4.944382,2.965108,130.260492,24.143867,122.366647,44.743932,5.080063,2.894163,128.251978,33.000693,101.725519,46.093510,0.850861,0.194308,0.646645,0.255673,0.497315,0.332444,0.894625,0.184803,124.794129,1.000000,0.283059,0.0
2,0.457010,0.001681,0.242888,0.212859,0.000000,0.000000,0.0,0.000000,0.000000,0.001756,0.0,0.000000,0.083055,0.018591,0.269798,39.353851,19.417332,19.936519,58.771183,109.543386,49.753490,59.789896,159.296876,54.642888,27.781965,26.860923,82.424853,41.242230,34.196356,7.045875,75.438586,109.592342,57.576640,52.015702,167.168982,53.470241,38.344391,15.125850,91.814632,5.049946,2.983966,163.576979,24.973042,109.544201,49.753659,5.078936,2.968023,162.268659,33.590792,109.597127,57.584515,0.918514,0.497554,0.747443,0.611432,0.633925,0.720923,0.920287,0.496596,94.948697,0.518740,0.419375,0.0
3,0.513244,0.005711,0.213781,0.251819,0.079795,0.000000,0.0,0.017594,0.007208,0.001288,0.0,0.000315,0.000000,0.000729,0.107022,46.322391,17.711957,28.610434,64.034349,116.075087,43.593124,72.481962,159.668211,51.430923,18.573016,32.857907,70.003940,40.365565,17.259087,23.106478,57.624652,102.641859,38.606995,64.034863,141.248854,50.805205,18.072101,32.733104,68.877306,5.177654,2.969214,156.242754,25.499379,116.081770,43.582671,5.071879,2.909002,158.343946,28.273928,102.648278,38.598300,0.951710,0.539286,0.855409,0.599998,0.618140,0.830304,0.964611,0.532073,74.221670,0.347202,0.361672,1.0
4,0.390319,0.009454,0.272884,0.373487,0.000000,0.000000,0.0,0.000000,0.000000,0.000196,0.0,0.000304,0.071442,0.026759,0.442831,37.552979,14.454975,23.098004,52.007953,101.044906,37.171973,63.872932,138.216879,53.971671,17.591669,36.380003,71.563340,40.717229,38.810699,1.906530,79.527927,97.446185,48.106253,49.339932,145.552438,51.733004,39.866956,11.866047,91.599960,4.978534,2.964

We need to reduce the feature dimensionality in order to run the algorithm more efficiently. Select the top 10 features

In [3]:
def getKBest(X, y, score_func=f_classif, k=10):
    k_best = SelectKBest(score_func=score_func, k=k).fit(X, y)

    idxs = k_best.get_support(indices=True)
    X = X.iloc[:,idxs]
    return X

In [4]:
def discretize_eqwidth(df, bins=3):
    for col in list(df):
        intervals = pd.cut(df[col], bins)
        df[col] = pd.cut(df[col], bins, labels=[str(i+1) for i in range(bins)])

        attrs = []
        values = intervals.unique().tolist()
        values.sort()

        for val in values:
            low, high = val.left, val.right
            attrs.append('{}:[{}, {}]'.format(col, low, high))

        lb = LabelBinarizer().fit_transform(df[col])
        if (len(attrs) == 2):
            v = list(map(lambda x: 1 - x, lb))
            lb = np.concatenate((lb, v), 1)

        df2 = pd.DataFrame(data=lb, columns=attrs)
        df = df.drop(columns=[col])
        df = pd.concat([df, df2], axis=1, join='inner')
    
    return df

def discretize_frequency(df, bins=3):
    for col in list(df):
        intervals = pd.qcut(df[col], bins, duplicates='drop')
        inte = intervals.value_counts().shape[0]
        df[col] = pd.qcut(df[col], bins, labels=[str(i+1) for i in range(inte)], duplicates='drop')
        attrs = []
        values = intervals.unique().tolist()
        values.sort()

        for val in values:
            low, high = val.left, val.right
            attrs.append('{}:[{}, {}]'.format(col, low, high))

        lb = LabelBinarizer().fit_transform(df[col])
        if (len(attrs) == 2):
            v = list(map(lambda x: 1 - x, lb))
            lb = np.concatenate((lb, v), 1)

        df2 = pd.DataFrame(data=lb, columns=attrs)
        df = df.drop(columns=[col])
        df = pd.concat([df, df2], axis=1, join='inner')
    
    return df
    

In [5]:
def getRules(X, sup, confidence=0.9, top=15):
    frequent_itemsets = apriori(X, min_support=sup, use_colnames=True)
    if frequent_itemsets.shape[0] == 0:
        return 0, 0, 0, 0
    
    rules = association_rules(frequent_itemsets, metric="confidence", min_threshold=confidence)
    num_rules = rules.shape[0]
    mean_lift = rules['lift'].mean()
    if frequent_itemsets.shape[0] > top:
        
        mean_lift_top = rules['lift'].sort_values(ascending=False)[:top].mean()
    else:
        mean_lift_top = mean_lift
        
    return rules, num_rules, mean_lift, mean_lift_top

In [6]:
def getTransformations(df, width_bins=[6,8], frequency=[3,5]):
    datasets = []
    
    X, y = df.drop(columns=['consensus']), df['consensus']
    X = getKBest(X, y)
    
    for bins in width_bins:
        X_disc = discretize_eqwidth(X.copy(), bins=bins)
        datasets.append((X_disc, 'Equal Width {}'.format(bins)))
        
    for bins in frequency:
        X_disc = discretize_frequency(X.copy(), bins=bins)
        datasets.append((X_disc, 'Equal Frequency {}'.format(bins)))
        
    return datasets

In [7]:
green_datasets = getTransformations(green)
sch_datasets = getTransformations(sch)
hin_datasets = getTransformations(hin)
col_datasets = getTransformations(col)

In [8]:
def evaluateSupport(datasets, filename='green'):
    measures = {}
    i = 0
    for df, name in datasets:
        supports = np.arange(0.15, 0.40, 0.02)
        for sup in supports:
            _, num_rules, mean_lift, mean_lift_top = getRules(df, sup, top=20)
            measures[i] = {'Dataset': filename, 'Discretization': name, 'Number of Rules': num_rules, 'Mean Lift': mean_lift, 'Support': sup}
            i += 1
            
    measures = pd.DataFrame.from_dict(measures, "index")
    measures.to_csv('../plot_data/assoc_rules/{}_arules.csv'.format(filename), index=False)
    

In [9]:
#evaluateSupport(green_datasets, filename='green')
#evaluateSupport(sch_datasets, filename='sch')
#evaluateSupport(hin_datasets, filename='hin')
evaluateSupport(col_datasets, filename='col')

In [15]:

rules, num_rules, mean_lift, mean_lift_top = getRules(col_datasets[3][0], 0.15, top=20)

In [29]:
rules.sort_values('lift', ascending=False)['antecedents'][16]

frozenset({'hsv_cervix_v_mean:[-0.001, 68.538]',
           'hsv_cervix_v_std:[-0.001, 33.421]'})

In [30]:
rules[['antecedents', 'consequents', 'support', 'lift']].sort_values('lift', ascending=False).to_csv('generated_assoc_rules.csv')